# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [6]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [7]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [3]:
import os
dir_list = os.listdir("test_images/")
print("There are {} files in this directory".format(len(dir_list)))
print(dir_list)

There are 6 files in this directory
['solidWhiteCurve.jpg', 'solidWhiteRight.jpg', 'solidYellowCurve.jpg', 'solidYellowCurve2.jpg', 'solidYellowLeft.jpg', 'whiteCarLaneSwitch.jpg']


run your solution on all test_images and make copies into the test_images directory).

In [100]:
print(image.shape)

def upper_bound(image):
    """
    Compute the upper bound of the trapezoid region that can 
    contain the road in proportion to the shape of the passed image.
    
    This way we can easily control 'how much road ahead' gets drawn.
    """
    return np.int(0.61 * image.shape[0])
    
def center_region(image):
    """
    Prepare coordinates of the central trapezoid region that will be 
    selected using the region_of_interest helper function.
    """
    y, x = image.shape[:2]
    return np.array([[
        (np.int(0.57 * x), np.int(0.61 * y)), # upper right
        (np.int(0.46 * x), np.int(0.61 * y)), # upper left
        (np.int(0.1458 * x), y),              # lower left
        (np.int(0.95 * x), y)                 # lower right
    ]], dtype=np.int32)


def left_region(image):
    """
    Prepare coordinates of the left part of the trapezoid region
    containing left part of the road. It will be used as one
    of the inputs of the region_of_interest helper function.
    """
    y, x = image.shape[:2]
    return np.array([[
        (x / 2, np.int(0.61 * y)),            # upper right
        (np.int(0.46 * x), np.int(0.61 * y)), # upper left  
        (np.int(0.1458 * x), y),              # lower left
        (x / 2, y)                            # lower right
    ]], dtype=np.int32)


def right_region(image):
    """
    Prepare coordinates of the left part of the trapezoid region
    containing right part of the road. It will be used as one
    of the inputs of the region_of_interest helper function.
    """
    y, x = image.shape[:2]
    return np.array([[
        (np.int(0.57 * x), np.int(0.61 * y)), # upper right
        (x / 2, np.int(0.61 * y)),            # upper left  
        (x / 2, y),                           # lower left
        (np.int(0.95 * x), y)                 # lower right
    ]], dtype=np.int32)


(720, 1280, 3)


In [162]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

def get_white_coordinates(img):
    """ Returns two numpy arrays with xs
    and ys of white pixels in given image"""
    assert len(img.shape) == 2, "Only meant for 2d images"
    acc = [(x, y) for y in range(img.shape[0]) 
           for x in range(img.shape[1]) 
           if img[y, x] == 255]
    xs = np.array([elem[0] for elem in acc])
    ys = np.array([elem[1] for elem in acc])
    return xs, ys


def fit_polynomial_line_extension(img_crop, degree=1):
    """ Based on input - image transformed by Canny edge 
    detection, fit a polynomial """
    # TODO(mike): Fix this comment
    xs, ys = get_white_coordinates(img_crop)

    fit_params = np.polyfit(ys, xs, degree)
    fit_f = np.poly1d(fit_params)

    # y_hat = np.arange(start=UPPER_BOUND, stop=img_gray.shape[0])
    # x_hat = fit_f(y_hat)
    # return x_hat, y_hat
    return fit_f
    

def process_image(image, eps=10, gauss_ksize=15, canny_low=50, canny_high=150):
    """
    """
    img_gray = grayscale(image)
    img_gaussian_blur = gaussian_blur(img_gray, gauss_ksize)  
    img_canny = canny(img_gaussian_blur, canny_low, canny_high)
    
    img_cropped = region_of_interest(img_canny, center_region(img_canny))
    img_cropped_right = region_of_interest(img_canny, right_region(img_canny))
    img_cropped_left = region_of_interest(img_canny, left_region(img_canny))
    
    mark_image = np.zeros_like(image)
    
    try:
        lf = fit_polynomial_line_extension(img_cropped_left)
        rf = fit_polynomial_line_extension(img_cropped_right)
        
        xx, yy = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
        r = eps ** 2
    
        left_selector = ((lf(yy) - xx) ** 2 <= r) & (yy > upper_bound(image))
        right_selector = ((rf(yy) - xx) ** 2 <= r) & (yy > upper_bound(image))
        mark_image[left_selector] = (255, 0, 0)
        mark_image[right_selector] = (255, 0, 0)
        
    except TypeError:
        print("Can't fit polynomial...")
    
    return weighted_img(mark_image, image)
    
    
for image_fname in dir_list:
    image = mpimg.imread(os.path.join('test_images', image_fname)) 
    img_name_prefix, img_name_extension = image_fname.split('.')
    processed_img_fname = os.path.join(
        'test_images',
        "{}_{}.{}".format(img_name_prefix, 'processed', img_name_extension)
    )
    processed_image = process_image(image)
    plt.imsave(fname=processed_img_fname, arr=processed_image)
    print("Saved {}".format(processed_img_fname))
    


Saved test_images/solidWhiteCurve_processed.jpg
Saved test_images/solidWhiteRight_processed.jpg
Saved test_images/solidYellowCurve_processed.jpg
Saved test_images/solidYellowCurve2_processed.jpg
Saved test_images/solidYellowLeft_processed.jpg
Saved test_images/whiteCarLaneSwitch_processed.jpg


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [163]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Let's try the one with the solid white lane on the right first ...

In [159]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

  self.nchannels))



Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [160]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [164]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

  self.nchannels))



In [165]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

My reflections are as follows:
* based on the optional challange, I got to appreciate that it is tough to build an algorithm that is robust. Simple variations in the image: changes of brightness / contrast / color balance can easily break an algorhitm
* a problem that we is perceptually simple for human (recognizing white stripes on black asphalt) can prove to be hard to code precisely using the standard tools (even the coming up with definition of white color is not a trivial task).
* if the environment is relatively stable, we can levarage relatively simple tools to solve the task of finding the road lines on the tarmac. Unfortunately 'real life' is not a 'relatively stable environment'
* I think the performance of the algorhitm can be improved significantly if we smooth the lines perceived over time, instead of treating the video as a series of images that are processed one by one.



## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [176]:
# extract samples from the harder clip
vid_fname = 'challenge.mp4'
clip = VideoFileClip(vid_fname)

snap_times = np.linspace(start=0, stop=clip.duration, num=25)

for idx, snap_t in enumerate(snap_times):
    clip.to_ImageClip(snap_t).save_frame('challenge_snaps/img_{}.png'.format(idx))


challenge_dir_list = os.listdir("challenge_snaps/")
print("There are {} files in this directory".format(len(challenge_dir_list)))
print(challenge_dir_list)


  self.nchannels))

  UserWarning)



There are 25 files in this directory
['img_0.png', 'img_1.png', 'img_10.png', 'img_11.png', 'img_12.png', 'img_13.png', 'img_14.png', 'img_15.png', 'img_16.png', 'img_17.png', 'img_18.png', 'img_19.png', 'img_2.png', 'img_20.png', 'img_21.png', 'img_22.png', 'img_23.png', 'img_24.png', 'img_3.png', 'img_4.png', 'img_5.png', 'img_6.png', 'img_7.png', 'img_8.png', 'img_9.png']


In [ ]:
# Diagnose performance of the algo on the excerpts of the harder part of the task
iidx = 11

# 9, 10, 11, 12 are the real hard part
raw_image = mpimg.imread(os.path.join('challenge_snaps', 'img_{}.png'.format(iidx)))
image = np.uint8(255 * raw_image)
# plt.imshow(process_image(image, gauss_ksize=9, canny_low=10, canny_high=125))

def test_process(image, gauss, canny_low, canny_high, show=True, show_blur=False):
    img_gray = grayscale(image)
    img_gaussian_blur = gaussian_blur(img_gray, gauss)  # best param currently
    img_canny = canny(img_gaussian_blur, canny_low, canny_high)
    img_cropped = region_of_interest(img_canny, center_region(img_canny))
    if show:
        plt.imshow(img_cropped, cmap='gray')
    if show_blur:
        plt.imshow(img_gaussian_blur, cmap='gray')
        
    return img_cropped

# test_process(image, 9, 10, 125, show=True, show_blur=False)

# loop over canny params
gauss_grid = [1, 3, 5, 7, 9, 11]
canny_grid = [5, 10, 20, 40, 50, 75, 100, 125, 150, 200, 300, 400, 500, 600]

def loop_over_params():
    import tqdm 
    for gauss in gauss_grid:
        print("Processing gauss = {}".format(gauss))
        for canny_low in tqdm.tqdm(canny_grid):
            for canny_high in canny_grid:
                if canny_low < canny_high:
                    nimg = process_image(image, gauss_ksize=gauss, canny_low=canny_low, canny_high=canny_high)
                    plt.imsave(
                        fname=os.path.join(
                            'canny_grid', 'canny_{}_{}_{}_{}.png'.format(iidx, gauss, canny_low, canny_high)
                        ),
                        arr=nimg, cmap='gray'
                    )

# loop_over_params()

# best currently 1, 75, 400
# also 1, 200, 400 also good
# 3, 75, 300
# 3, 100, 300
# 3, 150, 300
# 11, 75, 200
# 5, 75, 150
# 9, 50, 125
# 9, 100, 125 
# 11, 50, 100



  0%|          | 0/14 [00:00<?, ?it/s]

Processing gauss = 1



Exception in thread Thread-16:
Traceback (most recent call last):
  File "/Users/misiu-dev/anaconda3/envs/slfdrv-env/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/Users/misiu-dev/anaconda3/envs/slfdrv-env/lib/python3.5/site-packages/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/misiu-dev/anaconda3/envs/slfdrv-env/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  7%|▋         | 1/14 [01:03<13:48, 63.72s/it]

In [204]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(lambda img: process_image(img, eps=8, gauss_ksize=3, canny_low=150, canny_high=300))
%time challenge_clip.write_videofile(challenge_output, audio=False)

  self.nchannels))



[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


 45%|████▌     | 114/251 [06:22<06:06,  2.67s/it]

Can't fit polynomial...


 46%|████▌     | 115/251 [06:24<05:17,  2.33s/it]

Can't fit polynomial...


100%|██████████| 251/251 [1:23:55<00:00,  3.88s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 16min 7s, sys: 11.9 s, total: 16min 19s
Wall time: 1h 23min 56s


In [168]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))